https://developer.here.com/blog/getting-started-with-geocoding-exif-image-metadata-in-python3

https://reverse.geocoder.api.here.com/6.2/reversegeocode.json?prox=37.7442%2C-119.5931%2C1000&mode=
#retrieveAreas&app_id=devportal-demo-20180625&app_code=9v2BkviRwi9Ot26kp2IysQ&gen=9

In [1]:
# define functions to get geodata data . I got the instrutions from a website
import os
import requests

from PIL import Image

# funtion 1
def get_exif(filename):
    image = Image.open(filename)
    image.verify()
    return image._getexif()



from PIL.ExifTags import GPSTAGS
from PIL.ExifTags import TAGS
# funtion #2
def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging


#funtion 3
def get_decimal_from_dms(dms, ref):

    degrees = dms[0][0] / dms[0][1]
    minutes = dms[1][0] / dms[1][1] / 60.0
    seconds = dms[2][0] / dms[2][1] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)


# funtion 4

def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])

    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])

    return (lat,lon)


# Funtion 5

def get_location(geotags):
    coords = get_coordinates(geotags)

    uri = 'https://reverse.geocoder.api.here.com/6.2/reversegeocode.json'
    headers = {}
    params = {
        'app_id': 'cjIyVcZ1rzUhd0Xb9r0q',
        'app_code': 'f9pU60MvwiOVXVRl5N5HmA',
        'prox': "%s,%s" % coords,
        'gen': 9,
        'mode': 'retrieveAddresses',
        'maxresults': 1,
    }

    response = requests.get(uri, headers=headers, params=params)
    try:
        response.raise_for_status()
        return response.json()

    except requests.exceptions.HTTPError as e:
        print(str(e))
        return {}






In [2]:
# create a function to put the geotag data into a dictionary

def ExtractPicData(picAddress):
    PicData={'latitude':"",'longitude':'','landmark':'','country':'','state':'','county':'','city':'','zipcode':'','ImageTimeStamp':'','FileAddress':''}
    exif = get_exif(picAddress)
    geotags = get_geotagging(exif)
    location = get_location(geotags)

    latitude=location['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Latitude']
    longitude=location['Response']['View'][0]['Result'][0]['Location']['DisplayPosition']['Longitude']
    landmark=location['Response']['View'][0]['Result'][0]['Location']['Address']['Label']
    country=location['Response']['View'][0]['Result'][0]['Location']['Address']['Country']
    state=location['Response']['View'][0]['Result'][0]['Location']['Address']['State']
    county=location['Response']['View'][0]['Result'][0]['Location']['Address']['County']
    city=location['Response']['View'][0]['Result'][0]['Location']['Address']['City']
    zipcode=location['Response']['View'][0]['Result'][0]['Location']['Address']['PostalCode']
    ImageTimeStamp=location['Response']['View'][0]['Result'][0]['Location']['MapReference']['MapReleaseDate']
    FileAddress=picAddress

    PicData['latitude']=latitude
    PicData['longitude']=longitude
    PicData['landmark']=landmark
    PicData['country']=country
    PicData['state']=state
    PicData['county']=county
    PicData['city']=city
    PicData['zipcode']=zipcode
    PicData['ImageTimeStamp']=ImageTimeStamp
    PicData['FileAddress']="../"+FileAddress
    return PicData


In [3]:
# execute function

uploadsFolder="static/Images/uploads/"
picAddress=uploadsFolder +'test.jpg'
ReturnedData=ExtractPicData(picAddress)
# ReturnedData=PicData

# DataFrame=[]
# DataFrame.append(ReturnedData)

ReturnedData

{'latitude': 44.98036,
 'longitude': -93.26508,
 'landmark': 'Downtown West, Minneapolis, MN, United States',
 'country': 'USA',
 'state': 'MN',
 'county': 'Hennepin',
 'city': 'Minneapolis',
 'zipcode': '55401',
 'ImageTimeStamp': '2019-07-16',
 'FileAddress': '../static/Images/uploads/test.jpg'}

In [6]:
# get all the files in the images folder into a list called dirs

import os, sys
import pandas as pd
path = "static/images/"
dirs = os.listdir(path )

# Create an empty dictionary
PicturesDictionary=[]
# for each file in dirs execute extractPicData, check that the file is a jpg

for file in dirs:
    print(file)
    if file.find('jpg') is not -1:
        Picdata=path+file
        # Eexecute the function and insert the data retured by the function into a variable 
        ReturnedData=ExtractPicData(Picdata)
        # Append the data into the ditionary                 
        PicturesDictionary.append(ReturnedData)

# Convert the dictionary into a dataframe
PicturesDataFrame=pd.DataFrame(PicturesDictionary)   

PicturesDataFrame.tail(1)
#print(dirs)

away_1238.jpg
away_4732.JPG
away_5142.jpg
away_5731.JPG
away_5863.JPG
away_6258.JPG
blur_1261.jpg
blur_1380.jpg
blur_1382.jpg
blur_1495.jpg
blur_4660.jpg
crumple_140.jpg
crumple_254.JPG
crumple_4188.JPG
crumple_458.JPG
crumple_495.jpg
crumple_5427.JPG
crumple_6198.JPG
fun_3905.JPG
fun_4048.JPG
fun_4356.JPG
fun_5043.jpg
fun_5348.jpg
fun_5860.jpg
fun_6156.jpg
fun_6238.jpg
fun_808.JPG
fun_842.JPG
fun_E4173.jpg
garbage_288.jpg
garbage_514.jpg
garbage_64.jpg
garbage_854.jpg
gold_3710.JPG
gold_6480.JPG
gold_7527.jpg
leaves_0421.jpg
leaves_0484.jpg
leaves_1033.jpg
leaves_1325.JPG
leaves_1331.JPG
leaves_1378.jpg
leaves_1402.jpg
leaves_141.jpg
leaves_1449.jpg
leaves_1486.jpg
leaves_1490.jpg
leaves_1491.jpg
leaves_1492.jpg
leaves_1494.jpg
leaves_1503.jpg
leaves_1509.jpg
leaves_1511.jpg
leaves_1513.jpg
leaves_1522.jpg
leaves_1523.jpg
leaves_1527.jpg
leaves_1539.JPG
leaves_1586.jpg
leaves_2531.JPG
leaves_2784.JPG
leaves_3430.JPG
leaves_3731.JPG
leaves_3761.JPG
leaves_3763.JPG
leaves_3764.JPG
leave

,FileAddress,ImageTimeStamp,city,country,county,landmark,latitude,longitude,state,zipcode
0,../static/images/away_1238.jpg,2019-07-16,Dallas,USA,Dallas,"Historic District, Dallas, TX, United States",32.77828,-96.79485,TX,75201


In [7]:
PicturesDataFrame.columns

Index(['FileAddress', 'ImageTimeStamp', 'city', 'country', 'county',
       'landmark', 'latitude', 'longitude', 'state', 'zipcode'],
      dtype='object')

In [9]:
PicturesDataFrame.tail(1)

,FileAddress,ImageTimeStamp,city,country,county,landmark,latitude,longitude,state,zipcode
130,../static/images/wet_4463.jpg,2019-07-16,Chicago,USA,Cook,"Lincoln Park, Chicago, IL, United States",41.92184,-87.64744,IL,60602


In [8]:
# save to csv
# PicturesDataFrame.to_csv("db/OldportsGeoTagData.csv", index=True)
PicturesDataFrame.to_csv("db/OldportsGeoTagData.csv", index=False)



move and shrink file

In [12]:
def make_thumbnail(filename):
    img = Image.open(filename)

    (width, height) = img.size
    if width > height:
        ratio = 1200.0 / width
    else:
        ratio = 1200.0 / height

    img.thumbnail((round(width * ratio), round(height * ratio)), Image.LANCZOS)
    img.save(filename)


In [13]:
make_thumbnail('static/images/away_1238.jpg')

make_thumbnail('images/20190704_210738_Film3.jpg')

In [22]:

import os, sys
import pandas as pd
path = "static/images/"
dirs = os.listdir(path )

# Create an empty dictionary
PicturesDictionary=[]
# for each file in dirs execute extractPicData, check that the file is a jpg

for file in dirs:
    print(file)
    if file.find('jpg') is not -1:
        Picdata=path+file
        make_thumbnail(Picdata)
        # Eexecute the function and insert the data retured by the function into a variable 
       

away_4732.JPG
away_5731.JPG
away_6258.JPG
crumple_254.JPG
crumple_4188.JPG
crumple_458.JPG
crumple_5427.JPG
crumple_6198.JPG
fun_1287.jpg
fun_3905.JPG
fun_4048.JPG
fun_4356.JPG
fun_808.JPG
fun_842.JPG
gold_3710.JPG
gold_6480.JPG
IMG_3416.jpg
leaves_1325.JPG
leaves_1331.JPG
leaves_1539.JPG
leaves_2531.JPG
leaves_2784.JPG
leaves_3430.JPG
leaves_3731.JPG
leaves_3761.JPG
leaves_3763.JPG
leaves_3764.JPG
leaves_3820.JPG
leaves_4610.JPG
leaves_4656.JPG
leaves_4688.JPG
leaves_4689.JPG
leaves_4798.JPG
leaves_5191.JPG
leaves_5423.JPG
leaves_5737.JPG
leaves_5809.JPG
leaves_5849.JPG
leaves_5989.JPG
leaves_6013.JPG
leaves_6080.JPG
leaves_6121.JPG
leaves_6299.JPG
leaves_6459.JPG
leaves_6464.JPG
leaves_6472.JPG
leaves_6478.JPG
leaves_6479.JPG
leaves_6717.JPG
leaves_6718.JPG
leaves_8001.JPG
lush_1482.JPG
lush_1541.JPG
lush_1636.JPG
lush_1669.JPG
lush_3996.JPG
lush_6457.JPG
lush_7312.JPG
metal_1388.JPG
metal_1401.JPG
metal_3186.JPG
metal_4265.JPG
metal_5158.JPG
metal_6206.JPG
metal_7569.JPG
metal_8440.